In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [291]:
# import data
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/pytorch learning and project/liner regression from scratch/airfoil_self_noise.csv',header=None)

In [292]:
df.columns=['Freq','Angle','Chord_length','Free_stream','d_thickness','targets']
df.shape

(1503, 6)

In [293]:
df.head()

,Freq,Angle,Chord_length,Free_stream,d_thickness,targets
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [294]:
#check null values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.shape

(1503, 6)

In [295]:
import torch 
import numpy as np

In [296]:
# Input array
inputs = np.array(df.drop('targets', 1), dtype='float32')
targets = np.array(df['targets'], dtype='float32')

In [297]:
inputs.shape,targets.shape

((1503, 5), (1503,))

In [272]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[8.0000e+02, 0.0000e+00, 3.0480e-01, 7.1300e+01, 2.6634e-03],
        [1.0000e+03, 0.0000e+00, 3.0480e-01, 7.1300e+01, 2.6634e-03],
        [1.2500e+03, 0.0000e+00, 3.0480e-01, 7.1300e+01, 2.6634e-03],
        ...,
        [4.0000e+03, 1.5600e+01, 1.0160e-01, 3.9600e+01, 5.2849e-02],
        [5.0000e+03, 1.5600e+01, 1.0160e-01, 3.9600e+01, 5.2849e-02],
        [6.3000e+03, 1.5600e+01, 1.0160e-01, 3.9600e+01, 5.2849e-02]])
tensor([126.2010, 125.2010, 125.9510,  ..., 106.6040, 106.2240, 104.2040])


In [273]:
print(targets)

tensor([126.2010, 125.2010, 125.9510,  ..., 106.6040, 106.2240, 104.2040])


In [274]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3335, -1.6975,  0.4210],
        [ 0.6724,  0.5188, -0.5035]], requires_grad=True)
tensor([-1.0449, -0.7559], requires_grad=True)


In [275]:
df.columns

Index(['Freq', 'Angle', 'Chord_length', 'Free_stream', 'd_thickness',
       'targets'],
      dtype='object')

## Linear regression 


In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias.

Scaled sound pressure (outpt)= w11 * Freq + w12 * Angle + w13 * Chord_length + w14 * Free_stream + w15 * d_thickness + b1

## Initialize the weights & biase

In [276]:
# Weights and biases
w = torch.randn(1, 5, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(w)
print(b)

tensor([[-0.1302, -0.5869,  2.3823, -0.8717,  0.2931]], requires_grad=True)
tensor([1.3238], requires_grad=True)


torch.randn creates a tensor with the given shape, with elements picked randomly from a normal distribution with mean 0 and standard deviation 1

### Model

In [277]:
# model function 
def model(x):
    return x @ w.t() + b

In [278]:
# Generate predictions
preds = model(inputs)
print(preds)
print(targets)

tensor([[-164.2502],
        [-190.2872],
        [-222.8334],
        ...,
        [-562.8321],
        [-693.0168],
        [-862.2568]], grad_fn=<AddBackward0>)
tensor([126.2010, 125.2010, 125.9510,  ..., 106.6040, 106.2240, 104.2040])


### Loss function

The model prediction and actual outputs have a huge difference so we need a loss function minmize the error.

In [279]:
# MSE loss
def mse(t1, t2):
    t1=t1.squeeze()
    #print('t1',t1.shape)
    #print('t2',t2.shape)
    loss_fn = torch.nn.MSELoss()
    RMSE_loss = torch.sqrt(loss_fn(t1, t2))
    return RMSE_loss

In [280]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(683.0771, grad_fn=<SqrtBackward>)


### Compute gradients

In [232]:
# Compute gradients
loss.backward()

In [234]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.4475,  0.0515,  0.2133, -0.9735, -0.7686]], requires_grad=True)
tensor([[-4.2650e+03, -3.7642e+00, -9.8052e-02, -3.8202e+01, -5.9094e-03]])


### Adjust weights and biases to reduce the loss

In [235]:
print(w)
print(w.grad)

tensor([[-0.4475,  0.0515,  0.2133, -0.9735, -0.7686]], requires_grad=True)
tensor([[-4.2650e+03, -3.7642e+00, -9.8052e-02, -3.8202e+01, -5.9094e-03]])


We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

In [236]:
# learning arte --> 1e-5
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [ ]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)

Need to do gradients to zero bacause PyTorch accumulates gradients, otherwise next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [238]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0., 0., 0.]])
tensor([0.])


In [ ]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)

## Train the model using gradient descent

In [257]:
targets

tensor([126.2010, 125.2010, 125.9510,  ..., 106.6040, 106.2240, 104.2040])

In [286]:
# Train for 100 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    print('RMSE:',loss)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-6
        b -= b.grad * 1e-6
        w.grad.zero_()
        b.grad.zero_()


RMSE: tensor(12804.8984, grad_fn=<SqrtBackward>)
RMSE: tensor(12786.6338, grad_fn=<SqrtBackward>)
RMSE: tensor(12768.3691, grad_fn=<SqrtBackward>)
RMSE: tensor(12750.1045, grad_fn=<SqrtBackward>)
RMSE: tensor(12731.8398, grad_fn=<SqrtBackward>)
RMSE: tensor(12713.5742, grad_fn=<SqrtBackward>)
RMSE: tensor(12695.3086, grad_fn=<SqrtBackward>)
RMSE: tensor(12677.0439, grad_fn=<SqrtBackward>)
RMSE: tensor(12658.7783, grad_fn=<SqrtBackward>)
RMSE: tensor(12640.5137, grad_fn=<SqrtBackward>)
RMSE: tensor(12622.2500, grad_fn=<SqrtBackward>)
RMSE: tensor(12603.9844, grad_fn=<SqrtBackward>)
RMSE: tensor(12585.7188, grad_fn=<SqrtBackward>)
RMSE: tensor(12567.4541, grad_fn=<SqrtBackward>)
RMSE: tensor(12549.1895, grad_fn=<SqrtBackward>)
RMSE: tensor(12530.9238, grad_fn=<SqrtBackward>)
RMSE: tensor(12512.6602, grad_fn=<SqrtBackward>)
RMSE: tensor(12494.3945, grad_fn=<SqrtBackward>)
RMSE: tensor(12476.1299, grad_fn=<SqrtBackward>)
RMSE: tensor(12457.8652, grad_fn=<SqrtBackward>)
RMSE: tensor(12439.5

In [287]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(35.4040, grad_fn=<SqrtBackward>)


In [288]:
# Predictions # Targets
targets,preds

(tensor([126.2010, 125.2010, 125.9510,  ..., 106.6040, 106.2240, 104.2040]),
 tensor([[155.5817],
         [155.7352],
         [155.9272],
         ...,
         [ 93.9913],
         [ 94.7590],
         [ 95.7570]], grad_fn=<AddBackward0>))